In [1]:
import torch
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
import os
import random
import shutil

In [ ]:
import os
import random
import shutil

source = 'wikiart/Abstract_Expressionism'
dest = 'data/train/abstract-expressionism'
files = os.listdir(source)
no_of_files = 800

for file_name in random.sample(files, no_of_files):
    shutil.move(os.path.join(source, file_name), dest)

In [3]:
def move_shuffled_files(src, dest, num_files):
    files = os.listdir(src)
    for file_name in random.sample(files, num_files):
        shutil.move(os.path.join(src, file_name), dest)
    

In [8]:
def train_test_split():
    wiki_dirs = ['Early_Renaissance', 'Cubism', 'Pointillism', 'Realism']
    base_src = 'wikiart/'
    base_train_dest = 'data/train/'
    base_test_dest = 'data/validation/'
    for wiki_dir in wiki_dirs:
        src = base_src + wiki_dir
        train_dest = base_train_dest + wiki_dir
        test_dest = base_test_dest + wiki_dir
        move_shuffled_files(src, train_dest, 80)
        move_shuffled_files(src, test_dest, 20)
    
    
    

In [7]:
train_test_split()

In [24]:
train_transform = transforms.Compose([
                                      transforms.Resize([256, 256]),
                                      transforms.RandomHorizontalFlip(p=0.5),
                                      transforms.ColorJitter(0.3,0.3,0.3,0.3),
                                      transforms.RandomGrayscale(0.05),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5))
])

test_transform = transforms.Compose([
                                      transforms.Resize([256, 256]),
                                      transforms.ToTensor(),
])

In [25]:
train_data = datasets.ImageFolder(root="data/train", transform=train_transform)
train = DataLoader(train_data, batch_size = 4, shuffle = True)

In [26]:
test_data = datasets.ImageFolder(root="data/validation", transform=test_transform)
test = DataLoader(test_data, batch_size = 4, shuffle = False)

In [27]:
class CNNBlock(torch.nn.Module):
    def __init__(self, in_channels, out_channels, conv_kernel, conv_stride, conv_padding, pool_kernel, pool_stride):
        super().__init__()
        L = [
            torch.nn.BatchNorm2d(in_channels),
            torch.nn.ReLU(),
            torch.nn.Conv2d(in_channels, out_channels, conv_kernel, conv_stride, conv_padding, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None),
            torch.nn.BatchNorm2d(out_channels),
            torch.nn.SiLU(),
            torch.nn.MaxPool2d(pool_kernel, pool_stride),
        ]
        self.network = torch.nn.Sequential(*L)
    
    def forward(self, x):
        return self.network(x)

class CNNClassifier(torch.nn.Module):
    def __init__(self):
        super().__init__()
        in_channels = 3
        out_channels = 32
        num_classes = 4
        kernel_size = 7
        stride = 1
        padding = 3
        L = [
            CNNBlock(in_channels, out_channels, kernel_size, stride, padding, 2, 2),
            torch.nn.Dropout(p=0.05),
            CNNBlock(32, 64, 3, 1, 1, 2, 2),
            torch.nn.Dropout(p=0.1),
            CNNBlock(64, 128, 3, 1, 1, 2, 2),
            torch.nn.MaxPool2d(2),
            torch.nn.Flatten(),
            torch.nn.Linear(32768, num_classes)
        ]
        self.network = torch.nn.Sequential(*L)
        self.transforms = torchvision.transforms.Compose([torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    def forward(self, x):
        x = self.transforms(x)
        return self.network(x)

In [28]:
model = CNNClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = 0.01)

In [29]:
def train_iter(epoch, net, trainDataLoader, optimizer, criterion):
    net.train()
    total = 0
    train_loss = 0
    correct = 0

    for inputs, targets in trainDataLoader:
        optimizer.zero_grad()
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        loss = criterion(outputs,targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

    avg_loss = train_loss / len(trainDataLoader)
    accuracy = 100 * correct / total
    print('Epoch: {} \tTraining Loss: {:.25f} \tTraining Accuracy: {:.4f} %%'.format(epoch, avg_loss, accuracy))
    return avg_loss

In [30]:
def train_model(num_epochs, net, trainDataLoader, optimizer, criterion):
  for i in range(num_epochs):
    train_iter(i, net, trainDataLoader, optimizer, criterion)

In [31]:
train_model(5, model, train, optimizer, criterion)

Epoch: 0 	Training Loss: 57.2017701453296467661857605 	Training Accuracy: 33.4375
Epoch: 1 	Training Loss: 7.6091669380664823663096286 	Training Accuracy: 35.9375
Epoch: 2 	Training Loss: 4.8036403167992833829202937 	Training Accuracy: 38.4375
Epoch: 3 	Training Loss: 2.9725606070831416261057711 	Training Accuracy: 41.8750
Epoch: 4 	Training Loss: 2.9177090747980400919914246 	Training Accuracy: 40.3125


In [34]:
train_model(10, model, train, optimizer, criterion)

Epoch: 0 	Training Loss: 2.7318791658617556095123291 	Training Accuracy: 37.1875
Epoch: 1 	Training Loss: 2.9518452800810335290293551 	Training Accuracy: 43.7500
Epoch: 2 	Training Loss: 2.2851232238113881045649123 	Training Accuracy: 42.5000
Epoch: 3 	Training Loss: 2.4466302558779715603520799 	Training Accuracy: 41.2500
Epoch: 4 	Training Loss: 2.2878626094199718821187162 	Training Accuracy: 43.4375
Epoch: 5 	Training Loss: 2.3907029002904893744130277 	Training Accuracy: 43.1250
Epoch: 6 	Training Loss: 1.9155634072609246665308547 	Training Accuracy: 50.3125
Epoch: 7 	Training Loss: 3.0362857569009067404408597 	Training Accuracy: 40.6250
Epoch: 8 	Training Loss: 3.0607467183377594643900466 	Training Accuracy: 38.7500
Epoch: 9 	Training Loss: 4.1442793217487636781015681 	Training Accuracy: 43.1250


In [37]:
train_model(10, model, train, optimizer, criterion)

Epoch: 0 	Training Loss: 2.0226778466254473620722365 	Training Accuracy: 52.8125
Epoch: 1 	Training Loss: 3.1091960541380103677511215 	Training Accuracy: 45.6250
Epoch: 2 	Training Loss: 2.0291597127914426934580661 	Training Accuracy: 49.3750
Epoch: 3 	Training Loss: 2.4273703359998761719396043 	Training Accuracy: 45.6250
Epoch: 4 	Training Loss: 1.2099299445748328274419237 	Training Accuracy: 59.0625
Epoch: 5 	Training Loss: 1.5728334730491042581235206 	Training Accuracy: 54.6875
Epoch: 6 	Training Loss: 1.1024652587249874624575341 	Training Accuracy: 59.3750
Epoch: 7 	Training Loss: 1.0845762057229877406427931 	Training Accuracy: 61.2500
Epoch: 8 	Training Loss: 0.9724914727732538732851708 	Training Accuracy: 63.1250
Epoch: 9 	Training Loss: 1.1055432362947612556780541 	Training Accuracy: 60.6250


In [40]:
train_model(10, model, train, optimizer, criterion)

Epoch: 0 	Training Loss: 1.6421371965203435117075514 	Training Accuracy: 54.6875
Epoch: 1 	Training Loss: 1.0873040608596056255663598 	Training Accuracy: 62.1875
Epoch: 2 	Training Loss: 0.8935208981391042915021217 	Training Accuracy: 67.1875
Epoch: 3 	Training Loss: 1.1091279258951545205746925 	Training Accuracy: 60.9375
Epoch: 4 	Training Loss: 0.8411228692857548594474792 	Training Accuracy: 68.7500
Epoch: 5 	Training Loss: 0.8840034040389582736807483 	Training Accuracy: 70.6250
Epoch: 6 	Training Loss: 0.8285916869295760767144543 	Training Accuracy: 69.0625
Epoch: 7 	Training Loss: 0.6956357876770198345184326 	Training Accuracy: 77.8125
Epoch: 8 	Training Loss: 0.7178344917600043562089240 	Training Accuracy: 75.6250
Epoch: 9 	Training Loss: 0.8046942681772634164971691 	Training Accuracy: 70.0000


In [41]:
def eval_model(net, testDataLoader):
    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in testDataLoader:
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += len(targets)
            correct += (predicted == targets).sum().item()
            
    print('Accuracy of the network on the set of %d test images: %d %%' % (total,
        100 * correct / total))

In [42]:
eval_model(model, test)

Accuracy of the network on the set of 80 test images: 35 %
